In [14]:
import finrid_api_get as fapi
import db_functions as dbf
import pandas as pd

In [15]:
def get_df_db(conn, table, sql = None):
    curs = conn.cursor()
    sql = sql if sql else f'SELECT * FROM {table}'
    res = curs.execute(sql).fetchall()
    return res



In [16]:
DB_NAME = 'finnish_wind_power_orig.db'
CONN = dbf.create_connection(DB_NAME)


3.33.0


## Take data from API to db


In [50]:
wind_data = fapi.get_data_from_api(75)
wind_df = pd.json_normalize(wind_data)

consumption_data = fapi.get_data_from_api(124)
consumption_df = pd.json_normalize(consumption_data)

wind_df.to_sql('wind_power_generation', con=CONN, if_exists='append', index = False)
consumption_df.to_sql('electricity_consumption', con=CONN, if_exists='append', index = False)

OperationalError: database is locked

In [49]:

CONN.cursor().execute('DELETE FROM wind_power_generation')
CONN.cursor().execute('SELECT * FROM wind_power_generation').fetchall()


[]

In [43]:
get_df_db(conn, 'wind_power_generation')


[('2019-01-01T00:00:00+0000', '2019-01-01T01:00:00+0000', 1427),
 ('2019-01-01T01:00:00+0000', '2019-01-01T02:00:00+0000', 1352),
 ('2019-01-01T02:00:00+0000', '2019-01-01T03:00:00+0000', 1177),
 ('2019-01-01T03:00:00+0000', '2019-01-01T04:00:00+0000', 1001),
 ('2019-01-01T04:00:00+0000', '2019-01-01T05:00:00+0000', 826),
 ('2019-01-01T05:00:00+0000', '2019-01-01T06:00:00+0000', 642),
 ('2019-01-01T06:00:00+0000', '2019-01-01T07:00:00+0000', 423),
 ('2019-01-01T07:00:00+0000', '2019-01-01T08:00:00+0000', 284),
 ('2019-01-01T08:00:00+0000', '2019-01-01T09:00:00+0000', 247),
 ('2019-01-01T09:00:00+0000', '2019-01-01T10:00:00+0000', 259),
 ('2019-01-01T10:00:00+0000', '2019-01-01T11:00:00+0000', 346),
 ('2019-01-01T11:00:00+0000', '2019-01-01T12:00:00+0000', 596),
 ('2019-01-01T12:00:00+0000', '2019-01-01T13:00:00+0000', 720),
 ('2019-01-01T13:00:00+0000', '2019-01-01T14:00:00+0000', 856),
 ('2019-01-01T14:00:00+0000', '2019-01-01T15:00:00+0000', 941),
 ('2019-01-01T15:00:00+0000', '2019-